In [1]:
import tarfile

In [2]:
import json
import ast

In [3]:
import pandas as pd

In [4]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from scipy.sparse import coo_matrix as sp

C:\Users\Richa\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [5]:
my_tar = tarfile.open('C:/Users/Richa/Downloads/lthing_data.tar.gz')
my_tar.extractall('Ldata')
my_tar.close()

### Correcting JSON file

In [6]:
fr=open("C:/Users/Richa/Ldata/lthing_data/reviews.json")
fw=open("C:/Users/Richa/Ldata/lthing_data/reviews1.json", "w")

for line in fr:
    json_dat = json.dumps(ast.literal_eval(line))
    dict_dat = json.loads(json_dat)
    json.dump(dict_dat, fw)
    fw.write("\n")

fw.close()
fr.close()

In [7]:
with open('C:/Users/Richa/Ldata/lthing_data/reviews1.json') as json_file:
    data1 = json_file.readlines()
    data1 = list(map(json.loads, data1))
Lthing=pd.DataFrame(data1)
Lthing

,work,flags,unixtime,stars,nhelpful,time,comment,user
0,3206242,[],1.194394e+09,5.0,0,"Nov 7, 2007",This a great book for young readers to be intr...,van_stef
1,12198649,[],1.333757e+09,5.0,0,"Apr 7, 2012",Help Wanted: Tales of On The Job Terror from E...,dwatson2
2,12533765,[],1.352938e+09,NaN,0,"Nov 15, 2012","Magoon, K. (2012). Fire in the streets. New Yo...",edspicer
3,12981302,[],1.364515e+09,4.0,0,"Mar 29, 2013","Well, I definitely liked this book better than...",amdrane2
4,5231009,[],1.270944e+09,3.0,0,"Apr 11, 2010",It's a nice science-fiction thriller with some...,Lila_Gustavus
...,...,...,...,...,...,...,...,...
1707065,5377722,[],1.376438e+09,3.0,0,"Aug 14, 2013","Yea, so, this is borderline my type of thing. ...",Jellyn
1707066,13302111,[],1.356739e+09,NaN,0,"Dec 29, 2012",solito pacco editoriale natalizio di monologhi...,ShanaPat
1707067,452711,[],1.220227e+09,4.0,0,"Sep 1, 2008",In The Last Dive: A Father and Sons Fatal Desc...,koeniel
1707068,3109878,[],1.195690e+09,NaN,2,"Nov 22, 2007",The Age of Turbulence by Alan Greenspan\nA Rev...,Ductor


In [8]:
Lthing.to_csv('C:/Users/Richa/Ldata/lthing_data/reviews2.csv')

In [9]:
library=pd.read_csv('C:/Users/Richa/Ldata/lthing_data/reviews2.csv')

In [10]:
library.head()

,Unnamed: 0,work,flags,unixtime,stars,nhelpful,time,comment,user
0,0,3206242,[],1.194394e+09,5.0,0,"Nov 7, 2007",This a great book for young readers to be intr...,van_stef
1,1,12198649,[],1.333757e+09,5.0,0,"Apr 7, 2012",Help Wanted: Tales of On The Job Terror from E...,dwatson2
2,2,12533765,[],1.352938e+09,NaN,0,"Nov 15, 2012","Magoon, K. (2012). Fire in the streets. New Yo...",edspicer
3,3,12981302,[],1.364515e+09,4.0,0,"Mar 29, 2013","Well, I definitely liked this book better than...",amdrane2
4,4,5231009,[],1.270944e+09,3.0,0,"Apr 11, 2010",It's a nice science-fiction thriller with some...,Lila_Gustavus


In [11]:
library=library.drop(['Unnamed: 0','flags','unixtime','nhelpful','time','comment'],axis=1)
library.head()

,work,stars,user
0,3206242,5.0,van_stef
1,12198649,5.0,dwatson2
2,12533765,NaN,edspicer
3,12981302,4.0,amdrane2
4,5231009,3.0,Lila_Gustavus


### Correcting Group dataset

In [12]:
lib=pd.read_csv('C:/Users/Richa/Ldata/lthing_data/edges.txt',header=None)

In [13]:
lib.to_csv('C:/Users/Richa/Ldata/lthing_data/libgroup.csv')

In [14]:
data_1=pd.read_csv('C:/Users/Richa/Ldata/lthing_data/libgroup.csv')

In [15]:
data_1.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [16]:
data_1.columns=['SNo','Groupx']
data_1.columns

Index(['SNo', 'Groupx'], dtype='object')

In [17]:
data_2=data_1.copy(deep=True)

In [18]:
new_1 = data_1["Groupx"].str.split(" ", n = -1, expand = True)

In [19]:
new_2 = data_2["Groupx"].str.split(" ", n = -1, expand = True)

In [20]:
data_1["Group"]= new_1[0]
data_1["user"]= new_1[1]
data_1.drop(columns =["Groupx","SNo"], inplace = True)
data_1.head()

,Group,user
0,Rodo,anehan
1,Rodo,sevilemar
2,Rodo,dingsi
3,Rodo,slash
4,RelaxedReader,AnnRig


In [21]:
data_2["user"]= new_2[0]
data_2["Group"]= new_2[1]
data_2.drop(columns =["Groupx","SNo"], inplace = True)
data_2.head()

,user,Group
0,Rodo,anehan
1,Rodo,sevilemar
2,Rodo,dingsi
3,Rodo,slash
4,RelaxedReader,AnnRig


In [22]:
data = pd.concat([data_1,data_2])
data.head()

,Group,user
0,Rodo,anehan
1,Rodo,sevilemar
2,Rodo,dingsi
3,Rodo,slash
4,RelaxedReader,AnnRig


In [23]:
data.count()

Group    439580
user     439580
dtype: int64

In [24]:
#removing duplicate values
x = data.duplicated().sum()
print("No of Duplicate Values", x)
data.drop_duplicates(inplace=True)
y = data.duplicated().sum()
print("No of Duplicate Values after cleaning", y)

No of Duplicate Values 120394
No of Duplicate Values after cleaning 0


In [25]:
data.count()

Group    319186
user     319186
dtype: int64

In [26]:
data.to_csv('C:/Users/Richa/Ldata/lthing_data/libgroup.csv')

In [27]:
data=pd.read_csv('C:/Users/Richa/Ldata/lthing_data/libgroup.csv')
data.head()

,Unnamed: 0,Group,user
0,0,Rodo,anehan
1,1,Rodo,sevilemar
2,2,Rodo,dingsi
3,3,Rodo,slash
4,4,RelaxedReader,AnnRig


In [28]:
data=data.drop(['Unnamed: 0'],axis=1)

### Combining Tables

In [29]:
combinedTable1=pd.merge(library,data,on='user',how='left')

In [30]:
combinedTable1.head(15)

,work,stars,user,Group
0,3206242,5.0,van_stef,Mishalak
1,3206242,5.0,van_stef,davrich
2,3206242,5.0,van_stef,flemingt
3,3206242,5.0,van_stef,lisa211
4,3206242,5.0,van_stef,mouna
5,3206242,5.0,van_stef,CarolinaCatherine
6,3206242,5.0,van_stef,E59F
7,12198649,5.0,dwatson2,Jeff_Carlson
8,12198649,5.0,dwatson2,leeemerick
9,12533765,NaN,edspicer,MHSLibrary


In [31]:
#removing duplicate values
x1 = combinedTable1.duplicated().sum()
print("No of Duplicate Values", x)
combinedTable1.drop_duplicates(inplace=True)
y1 = combinedTable1.duplicated().sum()
print("No of Duplicate Values after cleaning", y)

No of Duplicate Values 120394
No of Duplicate Values after cleaning 0


In [32]:
combinedTable1.count()

work     27942847
stars    24100223
user     27942756
Group    27410835
dtype: int64

In [33]:
combinedTable1=combinedTable1.dropna()

In [34]:
combinedTable1.count()

work     23706309
stars    23706309
user     23706309
Group    23706309
dtype: int64

In [35]:
combinedTable1.to_csv('C:/Users/Richa/Ldata/lthing_data/combinedTable.csv')

In [36]:
combinedTable1=pd.read_csv('C:/Users/Richa/Ldata/lthing_data/combinedTable.csv')

In [37]:
data1=combinedTable1.copy(deep=True)

### Prepare the data

In [38]:
dataset=Dataset()

In [40]:
dataset.fit(data1.user.unique(), data1.work.unique())

In [41]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 25973, num_items 319214.


In [42]:
interactions, weights = dataset.build_interactions([tuple(i) for i in data1[['user','work','stars']].values])